In [24]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession as ps
import sqlite3

The strategy for cleaning the sports.sqlite database will be to go table by table and assess what needs to be cleaned. 

First, we need to turn every sql table into a Pandas DF.

In [34]:
conn = sqlite3.connect('./sports.sqlite') 
spark = ps.builder.getOrCreate()
          
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Player
                               ''', conn)
sql_query2 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Extra_Runs
                               ''', conn)
sql_query3 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Batting_Style
                               ''', conn)
sql_query4 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Bowling_style
                               ''', conn)
sql_query5 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Country
                               ''', conn)
sql_query6 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Season
                               ''', conn)
sql_query7 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM City
                               ''', conn)
sql_query8 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Outcome
                               ''', conn)
sql_query9 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Win_By
                               ''', conn)
sql_query10 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Wicket_Taken
                               ''', conn)
sql_query11 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Venue
                               ''', conn)
sql_query12 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Extra_Type
                               ''', conn)
sql_query13 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Out_Type
                               ''', conn)
sql_query14 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Toss_Decision
                               ''', conn)
sql_query15 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Umpire
                               ''', conn)
sql_query16 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Team
                               ''', conn)
sql_query17 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Ball_by_Ball
                               ''', conn)
sql_query18 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM sysdiagrams
                               ''', conn)
sql_query19 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM sqlite_sequence
                               ''', conn)
sql_query20 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Match
                               ''', conn)
sql_query21 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Rolee
                               ''', conn)
sql_query22 = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Player_Match
                               ''', conn)


player = pd.DataFrame(sql_query)
Extra_Runs = pd.DataFrame(sql_query2)
Batting_Style = pd.DataFrame(sql_query3)
Bowling_style = pd.DataFrame(sql_query4)
Country = pd.DataFrame(sql_query5)
Season = pd.DataFrame(sql_query6)
City = pd.DataFrame(sql_query7)
Outcome = pd.DataFrame(sql_query8)
Win_By = pd.DataFrame(sql_query9)
Wicket_Taken = pd.DataFrame(sql_query10)
Venue = pd.DataFrame(sql_query11)
Extra_Type = pd.DataFrame(sql_query12)
Out_Type = pd.DataFrame(sql_query13)
Toss_Decision = pd.DataFrame(sql_query14)
Umpire = pd.DataFrame(sql_query15)
Team = pd.DataFrame(sql_query16)
Ball_by_Ball = pd.DataFrame(sql_query17)
sysdiagrams = pd.DataFrame(sql_query18)
sqlite_sequence = pd.DataFrame(sql_query19)
Match = pd.DataFrame(sql_query20)
Roles = pd.DataFrame(sql_query21)
Player_Match = pd.DataFrame(sql_query22)

Each table will need to have it's foreign id columns dropped in favor of the data in those foreign tables starting with Player.

In [35]:
player.head()

,Player_Id,Player_Name,DOB,Batting_hand,Bowling_skill,Country_Name
0,1,SC Ganguly,1972-07-08 00:00:00,1,1.0,1
1,2,BB McCullum,1981-09-27 00:00:00,2,1.0,4
2,3,RT Ponting,1974-12-19 00:00:00,2,1.0,5
3,4,DJ Hussey,1977-07-15 00:00:00,2,2.0,5
4,5,Mohammad Hafeez,1980-10-17 00:00:00,2,2.0,6


In [36]:
player = player.drop(['Batting_hand', 'Country_Name'], axis=1)
player = player.dropna()
player

,Player_Id,Player_Name,DOB,Bowling_skill
0,1,SC Ganguly,1972-07-08 00:00:00,1.0
1,2,BB McCullum,1981-09-27 00:00:00,1.0
2,3,RT Ponting,1974-12-19 00:00:00,1.0
3,4,DJ Hussey,1977-07-15 00:00:00,2.0
4,5,Mohammad Hafeez,1980-10-17 00:00:00,2.0
...,...,...,...,...
462,463,T Shamsi,1990-02-18 00:00:00,8.0
463,464,SM Boland,1989-04-11 00:00:00,3.0
464,465,DL Chahar,1992-08-07 00:00:00,1.0
466,467,RV Pawar,1979-09-06 00:00:00,7.0


We can move on to the next table: Extra Runs

It seems this database will be consolidated into 4 tables. 1 for Matches and 1 for players 1 for seasons and 1 for teams. These 4 are necessary to ensure that no data is lost in storage and the database can be extracted in the same way if necessary.